# 1. Extract only Encoder in detail (for fine-tuning)

>## 1) Setting & Data loading

In [1]:
!pip install laion-clap
!pip install torch
!pip install torchvision

In [5]:
import numpy as np
import librosa
import laion_clap
import torch.nn as nn
import os

>## 2) Extract Audio Embedding
: 총 2000개 데이터 중 100개만 넣었음

In [6]:
import os

# 디렉토리 경로 지정
dir_path = "ESC-50-master/ESC-50-master/audio"

# 디렉토리 내 파일 목록 불러오기
file_list = os.listdir(dir_path)

# # 출력
# print(file_list[:5])
# print(len(file_list))

In [7]:
# prefix 추가
prefix = 'ESC-50-master/ESC-50-master/audio/'

audio_file = [prefix + item for item in file_list]

# print(audio_file[:10]) 
# print(len(audio_file))

In [8]:
# quantization
def int16_to_float32(x):
    return (x / 32767.0).astype(np.float32)


def float32_to_int16(x):
    x = np.clip(x, a_min=-1., a_max=1.)
    return (x * 32767.).astype(np.int16)

model = laion_clap.CLAP_Module(enable_fusion=False)
model.load_ckpt() # download the default pretrained checkpoint.


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load our best checkpoint in the paper.
Download completed!
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.weight 	 Loaded
aud

In [6]:
# Directly get audio embeddings from audio files

audio_embed = model.get_audio_embedding_from_filelist(x = audio_file[:50], use_tensor=False)
print(audio_embed[:,-20:])
print(audio_embed.shape)

[[ 2.33158506e-02  5.45929149e-02  7.16352537e-02  1.31517705e-02
   3.05129252e-02 -6.31129518e-02  1.49855511e-02  4.42864113e-02
  -4.53110784e-02 -6.76504523e-02 -1.26454774e-02 -5.57375327e-02
   3.73854488e-02  4.09765616e-02 -7.31308386e-02  2.02682018e-02
   1.49454279e-02 -3.83634567e-02  3.47014628e-02 -1.23500060e-02]
 [ 1.39670111e-02  1.38229467e-02 -2.11973232e-03 -1.91621892e-02
  -2.11460870e-02 -4.33889106e-02 -4.62117679e-02 -2.74212155e-02
  -1.58375371e-02 -4.47046012e-02 -2.58371942e-02  6.59703985e-02
   1.22593306e-01  2.69965883e-02  1.22840796e-03 -6.60994127e-02
  -3.86373177e-02 -4.40261047e-03 -3.93143203e-03  2.89975088e-02]
 [ 8.36510807e-02  1.01906650e-01  7.18332306e-02  6.55123964e-03
   6.47192495e-03 -2.05253921e-02  3.49606834e-02  4.27918509e-02
  -1.62978377e-02 -2.33648866e-02 -6.58885688e-02  6.45386847e-03
  -3.60516682e-02  3.38612171e-03 -3.21768709e-02 -1.69701669e-02
   4.25500944e-02  1.34484079e-02 -1.62685253e-02 -3.72272357e-03]
 [ 1.23

In [10]:
# import pickle
# with open('data/clap_audio_embeds.pkl', 'wb') as f:
#     pickle.dump((audio_embed), f)
# with open('data/clap_audio_embeds.pkl', 'rb') as f:
#     audio_embed = pickle.load(f)
# print(audio_embed.shape)

(50, 512)


>## 3) Extract Text Embedding
: 총 2000개 중 100개만 넣었음

In [9]:
import pandas as pd

# CSV 파일을 DataFrame으로 로드
df = pd.read_csv("ESC-50-master/ESC-50-master/meta/esc50.csv")

# 'category' column의 모든 데이터를 list로 받음
category_data = df['category'].tolist()

# 출력
print(len(category_data))
print(category_data[:10])

2000
['dog', 'chirping_birds', 'vacuum_cleaner', 'vacuum_cleaner', 'thunderstorm', 'thunderstorm', 'door_wood_knock', 'can_opening', 'crow', 'door_wood_knock']


In [10]:
# Get text embedings from texts:
text_data = category_data
text_embed = model.get_text_embedding(text_data[:50])
print(text_embed)
print(text_embed.shape)

[[ 0.00751023 -0.00121427  0.01298879 ...  0.02761818  0.00774936
   0.01292299]
 [ 0.04226422  0.08132149 -0.00011061 ... -0.02582695 -0.02809787
  -0.05834977]
 [ 0.00394155  0.0236528  -0.01777841 ...  0.05986715  0.03546726
   0.00757419]
 ...
 [ 0.06618918  0.07592878 -0.03068296 ...  0.01541831 -0.02546537
  -0.08116932]
 [ 0.06618918  0.07592878 -0.03068296 ...  0.01541831 -0.02546537
  -0.08116932]
 [-0.06413938  0.03432493 -0.01238014 ...  0.01617089 -0.00911491
  -0.07885502]]
(50, 512)


In [12]:
import pickle
with open('data/clap_text_embeds.pkl', 'wb') as f:
    pickle.dump((text_embed), f)
with open('data/clap_text_embeds.pkl', 'rb') as f:
    text_embed = pickle.load(f)
print(text_embed.shape)

(50, 512)


In [ ]:
# Compute cosine similarity
norm_text = np.linalg.norm(text_embed, axis=1, keepdims=True)
norm_audio = np.linalg.norm(audio_embed, axis=1, keepdims=True)
dot_product = np.dot(text_embed, audio_embed.T)
cos_similarity = dot_product / (norm_text * norm_audio.T)

print(cos_similarity)

In [ ]:
# ESC50_Model_Cosine_Similarlity [:100]

# Compute row-wise mean
row_means = np.mean(cos_similarity, axis=1)
# Compute overall mean
overall_mean = np.mean(row_means)

print("Overall mean cosine similarity:", overall_mean)

# 2. Pretrained Models

>## 1) Load Model (Dataset : ESC50)

In [16]:
import laion_clap
import glob
import json
import torch
import numpy as np

# device = torch.device('cuda:0')
# download https://drive.google.com/drive/folders/1scyH43eQAcrBz-5fAw44C6RNBhC3ejvX?usp=sharing and extract ./ESC50_1/test/0.tar to ./ESC50_1/test/

# Pick a Dataset type
esc50_test_dir = 'data/0.tar/*/' # (flac-json)
class_index_dict_path = 'label/ESC50_class_labels_indices_space.json'

# Load the model
model = laion_clap.CLAP_Module(enable_fusion=True) # device=device
model.load_ckpt()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [17]:
# Get the class index dict
class_index_dict = {v: k for v, k in json.load(open(class_index_dict_path)).items()}
print('class 갯수:',len(class_index_dict))
for key, value in list(class_index_dict.items())[:10]:
    print(key, value)

class 갯수: 50
dog 0
rooster 1
pig 2
cow 3
frog 4
cat 5
hen 6
insects 7
sheep 8
crow 9


>## 2) Load Audio / Text / Ground-truth files

In [ ]:
# Get all the data
import tarfile

# 압축 파일 해제
# with tarfile.open('data/0.tar') as tar:
#     tar.extractall('data/ESC50_1/test/')

# # flac 파일 경로 가져오기
audio_files = sorted(glob.glob('data/ESC50_1/test/**/*.flac', recursive=True))

print('audio files 갯수:',len(audio_files)) # 여러개 파일 
print(audio_files[:3])

In [ ]:
json_files = sorted(glob.glob('./data/ESC50_1/test/**/*.json', recursive=True))
print('json_files 갯수:',len(json_files))
print(json_files[:3])

In [ ]:
ground_truth_idx = [class_index_dict[json.load(open(jf))['category']] for jf in json_files]
print('ground_truth_idx 갯수:',len(ground_truth_idx))
print(ground_truth_idx[:20])

>## 3) Zeroshot Classification

In [ ]:
import torchaudio

torchaudio.__version__

>>### (1) Audio/Text Embedding 값 불러오기 & 간단스크립트

In [ ]:
with torch.no_grad():
    ground_truth = torch.tensor(ground_truth_idx).view(-1, 1)

    # Get text features
    all_texts = ["This is a sound of " + t for t in class_index_dict.keys()]
    text_embed = model.get_text_embedding(all_texts)
    audio_embed = model.get_audio_embedding_from_filelist(x=audio_files)
   

- ground_truth_idx 리스트(classification 답)를 PyTorch 텐서로 변환하고,.view로 차원을 맞춰둠.
- all texts 인자에 class_index_dict의 key값(text)를 받아 확인할 수 있게 하고,
- text인 all texts 리스트에 대한 임베딩 값 text_embed 얻고,
- audio files 리스트에 있는 모든 오디오 파일에 대한 audio_embed 얻음.


In [ ]:
    print('text_embed length :',len(text_embed)) # class 갯수
    print('text_embed shape :',text_embed.shape)
    print(text_embed[:1])

In [ ]:
    print('audio_embed length :',len(audio_embed)) # 오디오파일 갯수
    print('audio_embed shape :',audio_embed.shape)
    print(audio_embed[:1])

>>### (2) Contrastive Learning & Prediction

In [ ]:
    ranking = torch.argsort(torch.tensor(audio_embed) 
                            @ torch.tensor(text_embed).t(), 
                            descending=True)
    

1. audio_embed & text_embed.T 내적곱해서 latent space 통일
2. torch.argsort : 계산한 내적값 작은순으로 sort해서 다시 index 부여
3. 다시 내림차순으로 정렬  
**>>> ranking = 음성과 텍스트 데이터간 유사도가 높은 순서대로 정렬된 인덱스가 담김**    

In [ ]:
    preds = torch.where(ranking == ground_truth)[1]
    preds = preds.cpu().numpy()

1. 가장 유사하다고 판단한 ranking 값과 실제답 ground_truth가 같으면 true를 반환함
2. true값의 위치를 찾아 주는 것이 torch.where
3.  [1] = 인덱스값   
**>>> preds = 음성과 텍스트간의 유사도가 가장 높은 class name(text)를 담음**
4. tensor를 np로 변환

>>### (3) Results

In [ ]:
    metrics = {}
    metrics[f"mean_rank"] = preds.mean() + 1
    metrics[f"median_rank"] = np.floor(np.median(preds)) + 1

In [ ]:
    print(metrics[f"mean_rank"])
    print(metrics[f"median_rank"])

1. 결과값을 담을 metrics dictionary 열어두고
2. preds의 평균값과 중앙값을 구해서 순위를 0이 아닌 1부터 시작하게함

In [ ]:
    for k in [1, 5, 10]:
        metrics[f"R@{k}"] = np.mean(preds < k)
    # map@10
    metrics[f"mAP@10"] = np.mean(np.where(preds < 10, 1 / (preds + 1), 0.0))

- R@1,5,10 = 예측된 순위(preds)가 실상위 k개 안에 있으면 1, 아니면 0 한 뒤 평균
- mAP@10 = Mean Average Precision at 10 : 모델이 상위 10개 예측에 대해 얼마나 정확한 결과를 내는지 평가하는 지표
    

In [ ]:
    print(
        f"Zeroshot Classification Results: "
        + "\t".join([f"{k}: {round(v, 4):.4f}" for k, v in metrics.items()])
    )

- mean_rank: 1.2425 : 모델이 예측한 결과 중 상위 2개 클래스가 정답 클래스로 예측되었음.
- median_rank: 1.0 : 중앙값으로 예측해도 평균적으로 1이라는 것은 대체로 예측된 클래스 중 1위 클래스가 실제값이라는 뜻.
- R@1,5,10 : 예측한 1,5,10위까지의 순위 안에 정답이 포함될 확률

        

>>### (4) Visualization

In [ ]:
print(type(ranking))
print(type(ground_truth))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

def confusion_matrix(y_true, y_pred, num_classes):
    # num_classes x num_classes 크기의 0으로 채워진 행렬 생성
    conf_mat = np.zeros((num_classes, num_classes))

    # 행렬의 각 [i][j] 요소는 i번 클래스가 j번 클래스로 분류된 샘플의 수
    for i in range(len(y_true)):
        true_idx = y_true[i]
        pred_idx = y_pred[i]
        conf_mat[true_idx][pred_idx] += 1

    return conf_mat

def plot_confusion_matrix(conf_mat, class_names):
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(8, 8))
    disp = disp.plot(include_values=True, cmap='Blues', ax=ax, xticks_rotation=90)  # x축 레이블을 90도 기울임
    plt.show()

# confusion matrix 생성
num_classes = len(class_index_dict)
conf_mat = confusion_matrix(ground_truth, torch.argmax(ranking, dim=1), num_classes)

# 클래스 이름 가져오기
class_names = list(class_index_dict.keys())

# 시각화
plot_confusion_matrix(conf_mat, class_names)


hand saw - dog(8) / cat - hand saw(6) / chainsaw - rooster(6) 

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

wav_path = './data/dog.wav'
audio = AudioSegment.from_file(wav_path, format="wav")
play(audio)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

def confusion_matrix(y_true, y_pred, num_classes):
    # num_classes x num_classes 크기의 0으로 채워진 행렬 생성
    conf_mat = np.zeros((num_classes, num_classes))

    # 행렬의 각 [i][j] 요소는 i번 클래스가 j번 클래스로 분류된 샘플의 수
    for i in range(len(y_true)):
        true_idx = y_true[i]
        pred_idx = y_pred[i]
        conf_mat[true_idx][pred_idx] += 1

    return conf_mat

def plot_confusion_matrix(conf_mat, class_names):
    fig, ax = plt.subplots()
    im = ax.imshow(conf_mat, interpolation='nearest', cmap='Blues')
    
    # x, y 축 설정
    ax.set(xticks=np.arange(conf_mat.shape[1]), yticks=np.arange(conf_mat.shape[0]),
           xticklabels=class_names, yticklabels=class_names)
    
    # x 축 레이블 90도 회전
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")
    
    # 각 셀에 값 출력
    for i in range(conf_mat.shape[0]):
        for j in range(conf_mat.shape[1]):
            text = ax.text(j, i, conf_mat[i, j], ha="center", va="center", color="w")
    
    # colorbar 추가
    cbar = ax.figure.colorbar(im, ax=ax)
    
    # 그래프 보여주기
    fig.tight_layout()
    plt.show()

# confusion matrix 생성
num_classes = len(class_index_dict)
conf_mat = confusion_matrix(ground_truth, torch.argmax(ranking, dim=1), num_classes)

# 클래스 이름 가져오기
class_names = list(class_index_dict.keys())

# 시각화
plot_confusion_matrix(conf_mat, class_names)


In [ ]:
stop

In [ ]:
num_classes = len(class_index_dict)
conf_mat = confusion_matrix(ground_truth, ranking, num_classes)


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

# 오차행렬 생성
conf_mat = multilabel_confusion_matrix(ground_truth, ranking)

# 클래스 이름 가져오기
class_names = list(class_index_dict.keys())

# 시각화
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 예측값과 실제값이 일치하는지 계산
correct_preds = (ranking == ground_truth)
print(correct_preds.shape)
incorrect_preds = (ranking != ground_truth)
incorrect_ranking = ranking[incorrect_preds][0]
print(incorrect_preds.shape)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 오차행렬 생성
conf_mat = confusion_matrix(ground_truth[incorrect_preds], ranking[incorrect_preds])

# 클래스 이름 가져오기
class_names = list(class_index_dict.keys())

# 오차행렬 시각화
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap=plt.cm.Blues, cbar=False, ax=ax)

# x축, y축 라벨 설정
ax.set_xlabel('Predicted labels', fontsize=16)
ax.set_ylabel('True labels', fontsize=16)
ax.set_xticklabels(class_names, fontsize=12)
ax.set_yticklabels(class_names, fontsize=12)

# 그래프 제목 설정
ax.set_title('Confusion Matrix', fontsize=20)
plt.show()


In [ ]:
stop

In [ ]:
# 일치하는 경우 0, 그렇지 않은 경우 1 array 만들기
color_indices = np.where(correct_preds, 0, 1)
color_indices

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(30, 100))
im = ax.imshow(color_indices, cmap="viridis")

ax.set_xticks(np.arange(color_indices.shape[1]))
ax.set_yticks(np.arange(color_indices.shape[0]))
ax.set_xticklabels(class_index_dict, fontsize=8)
ax.set_yticklabels(range(400), fontsize=8)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.title("Confusion matrix")
plt.show()


In [ ]:
class_index_dict

In [ ]:
ranking

In [ ]:
a= [class_index_dict[i] for i in preds]
a

In [ ]:
import pandas as pd
import json

class_index_dict_path = 'data/ESC50_class_labels_indices_space.json'
class_index_dict = {v: k for v, k in json.load(open(class_index_dict_path)).items()}

df = pd.DataFrame({
    'preds': preds,
    'ground_truth': ground_truth.squeeze().tolist(),
    'audio_file': audio_files,
    'actual_class': [class_index_dict[i] for i in ground_truth.squeeze().tolist()],
    'predicted_class': [class_index_dict[i] for i in preds]
})

df_diff = df[df['actual_class'] != df['predicted_class']]
display(df_diff.head())
